# 채팅 모델로 함수를 호출하는 방법

이 노트북에서는 채팅 완료 API를 외부 함수와 함께 사용하여 GPT 모델의 기능을 확장하는 방법을 다룹니다.

'함수'는 채팅 완료 API의 선택적 파라미터로 함수 사양을 제공하는 데 사용할 수 있습니다. 그 목적은 모델이 제공된 사양을 준수하는 함수 인수를 생성할 수 있도록 하기 위함입니다. API가 실제로 함수 호출을 실행하지는 않는다는 점에 유의하세요. 모델 출력을 사용하여 함수 호출을 실행하는 것은 개발자의 몫입니다.

'함수' 매개변수가 제공되면 기본적으로 모델이 함수 중 하나를 사용하는 것이 적절한 시기를 결정합니다. API는 `function_call` 파라미터를 `{"name": "<인서트 함수 이름>"}`로 설정할 수 있습니다. 또한 `function_call` 파라미터를 `"none"`으로 설정하여 API가 함수를 사용하지 않도록 강제할 수도 있습니다. 함수가 사용된 경우 출력에 `"finish_reason"`이 포함됩니다: "function_call"`과 함께 함수 이름과 생성된 함수 인수가 포함된 `function_call` 객체가 응답에 포함됩니다.

### 개요

이 노트북은 다음 2개의 섹션으로 구성되어 있습니다:

- 함수 인수를 생성하는 방법:** 함수 집합을 지정하고 API를 사용하여 함수 인수를 생성합니다.
- 모델 생성 인수를 사용하여 함수를 호출하는 방법:** 모델 생성 인수를 사용하여 함수를 실제로 실행하여 루프를 닫습니다.

함수 인수를 생성하는 방법 ##

In [ ]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor 
!pip install openai
!pip install requests

In [1]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"

### 유틸리티

먼저 채팅 완료 API를 호출하고 대화 상태를 유지 및 추적하기 위한 몇 가지 유틸리티를 정의해 보겠습니다.

In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [3]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### 기본 개념

가상의 날씨 API와 인터페이스하기 위한 몇 가지 함수 사양을 만들어 보겠습니다. 이 함수 사양을 채팅 완성 API에 전달하여 사양을 준수하는 함수 인수를 생성하도록 하겠습니다.

In [4]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
    {
        "name": "get_n_day_weather_forecast",
        "description": "Get an N-day weather forecast",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
                "num_days": {
                    "type": "integer",
                    "description": "The number of days to forecast",
                }
            },
            "required": ["location", "format", "num_days"]
        },
    },
]

모델에게 현재 날씨에 대해 질문하면 몇 가지 명확한 질문으로 응답합니다.

In [5]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': 'In which city and state would you like to know the current weather?'}

누락된 정보를 제공하면 적절한 함수 인수를 생성합니다.

In [6]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}'}}

프롬프트를 다르게 표시하면 앞서 설명한 다른 기능을 타겟팅하도록 할 수 있습니다.

In [7]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': 'Sure, I can help you with that. Please provide me with the number of days you want to forecast for.'}

다시 한 번, 모델에 아직 충분한 정보가 없기 때문에 설명을 요청하고 있습니다. 이 경우 예보의 위치는 이미 알고 있지만 예보에 필요한 일수를 알아야 합니다.

In [8]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, functions=functions
)
chat_response.json()["choices"][0]


{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_n_day_weather_forecast',
   'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}'}},
 'finish_reason': 'function_call'}

#### 특정 기능 사용 강제 또는 기능 없음

function_call 인수를 사용하여 모델이 특정 함수(예: get_n_day_weather_forecast)를 사용하도록 강제할 수 있습니다. 이렇게 하면 모델이 함수를 사용하는 방법에 대한 가정을 하도록 강제할 수 있습니다.

In [9]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, functions=functions, function_call={"name": "get_n_day_weather_forecast"}
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_n_day_weather_forecast',
  'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}'}}

In [10]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, functions=functions
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}'}}

모델이 함수를 전혀 사용하지 않도록 강제할 수도 있습니다. 이렇게 하면 적절한 함수 호출을 생성하지 못하게 됩니다.

In [11]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, functions=functions, function_call="none"
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant', 'content': 'Sure, let me get that information for you.'}

모델 생성 인자로 함수를 호출하는 방법 ## 모델 생성 인자로 함수를 호출하는 방법

다음 예제에서는 입력이 모델 생성된 함수를 실행하고 이를 사용하여 데이터베이스에 대한 질문에 답할 수 있는 에이전트를 구현하는 방법을 보여드리겠습니다. 간단하게 하기 위해 [치누크 샘플 데이터베이스](https://www.sqlitetutorial.net/sqlite-sample-database/)를 사용하겠습니다.

*참고: *모델이 올바른 SQL을 생성하는 데 완벽하게 신뢰할 수 없기 때문에 프로덕션 환경에서는 SQL 생성의 위험이 높을 수 있습니다.

### SQL 쿼리 실행을 위한 함수 지정하기

먼저 SQLite 데이터베이스에서 데이터를 추출하는 데 유용한 몇 가지 유틸리티 함수를 정의해 보겠습니다.

In [12]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

Opened database successfully


In [13]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


이제 이러한 유틸리티 함수를 사용하여 데이터베이스 스키마의 표현을 추출할 수 있습니다.

In [14]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

이전과 마찬가지로 API가 인수를 생성하도록 할 함수에 대한 함수 사양을 정의하겠습니다. 함수 사양에 데이터베이스 스키마를 삽입하고 있다는 점에 주목하세요. 이는 모델이 알아야 할 중요한 정보입니다.

In [15]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

SQL 쿼리 실행 중 ###

이제 데이터베이스에 대해 실제로 쿼리를 실행하는 함수를 구현해 보겠습니다.

In [16]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message["function_call"]["name"] == "ask_database":
        query = json.loads(message["function_call"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results

In [17]:
messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})
chat_response = chat_completion_request(messages, functions)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant: {'name': 'ask_database', 'arguments': '{\n  "query": "SELECT ar.Name, COUNT(t.TrackId) AS NumTracks FROM Artist ar INNER JOIN Album al ON ar.ArtistId = al.ArtistId INNER JOIN Track t ON al.AlbumId = t.AlbumId GROUP BY ar.ArtistId ORDER BY NumTracks DESC LIMIT 5"\n}'}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]



In [18]:
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})
chat_response = chat_completion_request(messages, functions)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant: {'name': 'ask_database', 'arguments': '{\n  "query": "SELECT ar.Name, COUNT(t.TrackId) AS NumTracks FROM Artist ar INNER JOIN Album al ON ar.ArtistId = al.ArtistId INNER JOIN Track t ON al.AlbumId = t.AlbumId GROUP BY ar.ArtistId ORDER BY NumTracks DESC LIMIT 5"\n}'}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]

user: What is the name of the album with the most tracks?

assistant: {'name': 'ask_database', 'arguments': '{\n  "query": "SELECT al.Title, COUNT(t.TrackId) AS NumTracks FROM Album al INNER JOIN Track t ON al.AlbumId = t.AlbumId GROUP BY al.AlbumId ORDER BY NumTracks DESC LIMIT 1"\n}'}

function (ask_database): [('Greatest Hits', 57)]



## 다음 단계

지식창고와 대화식으로 상호작용하기 위해 채팅 완료 API 및 지식 검색용 함수를 사용하는 방법을 설명하는 다른 [노트북](How_to_call_functions_for_knowledge_retrieval.ipynb)을 참조하세요.